In [1]:
pwd = %pwd

import os, sys
sys.path.insert(0, os.path.join(pwd, "../"))


#

-------

自作の強化学習アルゴリズムを作成するには以下のクラスを定義します。  
  
RLConfig  
RLRemoteMemory  
RLParameter  
RLTrainer  
RLWorker  

ここでは一番簡単なテーブル型のQ学習を自作してみます。

In [2]:
from typing import Any, List, Tuple, cast


# RLConfig

---
強化学習アルゴリズムの種類やハイパーパラメータ等を管理するクラスです。  
RLConfigを継承して作成してもいいですが、強化学習の種類によるインタフェースも提供しています。  
'srl.rl.algorithms'

In [3]:
from dataclasses import dataclass
from srl.base.rl.algorithms.table import TableConfig

@dataclass
class MyConfig(TableConfig):
    
    # ハイパーパラメータ
    epsilon: float = 0.1
    test_epsilon: float = 0
    gamma: float = 0.9
    lr: float = 0.1

    # 名前だけユニークなものを指定
    @staticmethod
    def getName() -> str:
        return "MyRL"


# RemoteMemory

---
Workerが取得した情報(経験)をTrainerに渡す役割を持っているクラスです。  
分散学習では multiprocessing のサーバプロセス(Manager)になります。  
ですので、変数へのアクセスができなくなる点だけ制約があります。（全て関数経由でやり取りする必要があります）  
  

In [4]:
from srl.base.rl.base import RLRemoteMemory

class MyRemoteMemory(RLRemoteMemory):

    # init の引数は親クラスにそのまま渡します。
    def __init__(self, *args):
        super().__init__(*args)
        # self.config は上で定義した MyConfig が入っています
        self.config = cast(MyConfig, self.config)

        # 経験を一時的に保存する用
        self.buffer = []

    # (abstractmethod) メモリに保存されている数を返す
    def length(self) -> int:
        return len(self.buffer)

    # (abstractmethod) restore/backupで復元できるように作成
    def restore(self, data: Any) -> None:
        self.buffer = data

    def backup(self):
        return self.buffer

    # --- 以下は独自に定義している関数です

    def add(self, batch: Any) -> None:
        self.buffer.append(batch)

    def sample(self):
        buffer = self.buffer
        self.buffer = []
        return buffer


またよくあるクラスは 'srl.rl.remote_memoty' に定義しています。  
経験を順番通りに取り出す SequenceRemoteMemory を使う場合は以下です。  


In [5]:
from srl.rl.remote_memory.sequence_memory import SequenceRemoteMemory

class MyRemoteMemory(SequenceRemoteMemory):
    pass


# RLParameter

---
学習する/したパラメータを管理するクラスです。  
ある意味強化学習の本体でもあります。  


In [6]:
import json
from srl.base.rl.base import RLParameter

import numpy as np

class MyParameter(RLParameter):

    # init の引数は親クラスにそのまま渡します。
    def __init__(self, *args):
        super().__init__(*args)
        # self.config は上で定義した MyConfig が入っています
        self.config = cast(MyConfig, self.config)

        # Q学習用のテーブル
        self.Q = {}

    # (abstractmethod) restore/backupでQテーブルを復元できるように作成
    def restore(self, data: Any) -> None:
        self.Q = json.loads(data)

    def backup(self):
        return json.dumps(self.Q)

    # Q値を取得する関数
    def get_action_values(self, state: np.ndarray, invalid_actions, to_str: bool = True):
        if to_str:
            state = str(state.tolist())
        if state not in self.Q:
            self.Q[state] = [-np.inf if a in invalid_actions else 0 for a in range(self.config.nb_actions)]
        return self.Q[state]


# RLWorker

---
経験を収集するクラスです。  
実際に環境と連携して動作する部分にもなります。  
役割は、Parameterを参照してアクションを決める事と、経験をMemoryに送信する事です。  

RLWorkerをそのまま継承しても問題ありませんが、  
作成するアルゴリズムによりインタフェースが変わるので、  
アルゴリズムに合わせたクラスを継承しても問題ありません。  


In [7]:
import random
from srl.base.rl.algorithms.table import TableWorker
from srl.base.env.env_for_rl import EnvForRL

class MyWorker(TableWorker):

    # init の引数は親クラスにそのまま渡します。
    def __init__(self, *args):
        super().__init__(*args)

        # 上で定義した config,parameter,remote_memory が入ります
        self.config = cast(MyConfig, self.config)
        self.parameter = cast(MyParameter, self.parameter)
        self.remote_memory = cast(MyRemoteMemory, self.remote_memory)

    # エピソードの最初に呼ばれる関数
    # state: 環境の初期状態が入っています
    # invalid_actions: 有効でないアクションのリストです
    def call_on_reset(self, state: np.ndarray, invalid_actions: List[int]) -> None:
        if self.training:
            self.epsilon = self.config.epsilon
        else:
            self.epsilon = self.config.test_epsilon

    # 戦略部分です。アクションを返します。
    # state: 環境の状態が入っています
    # invalid_actions: 有効でないアクションのリストです
    # 戻り値は2つとり、1番目は環境に渡すアクション、2番目はon_stepに渡されるアクションになります
    def call_policy(self, state: np.ndarray, invalid_actions: List[int]) -> Tuple[int, Any]:

        if random.random() < self.epsilon:
            # epsilonより低いならランダムに移動
            action = random.choice([a for a in range(self.config.nb_actions) if a not in invalid_actions])
        else:
            q = self.parameter.get_action_values(state, invalid_actions)
            q = np.asarray(q)

            # 最大値を選ぶ（複数あればランダム）
            action = random.choice(np.where(q == q.max())[0])

        return action, action

    # 1step毎に呼ばれる関数です
    # state     : アクション実行前の状態
    # action    : policyの2番目のアクション
    # next_state: アクション実行後の状態
    # reweard   : アクション実行後の報酬
    # done      : アクション実行後の終了状態
    # invalid_actions     : アクション実行前の有効でないアクションのリスト
    # next_invalid_actions: アクション実行後の有効でないアクションのリスト
    def call_on_step(
        self,
        state: np.ndarray,
        action: int,
        next_state: np.ndarray,
        reward: float,
        done: bool,
        invalid_actions: List[int],
        next_invalid_actions: List[int],
    ):
        if not self.training:
            return {}
        
        batch = {
            "state": str(state.tolist()),
            "next_state": str(next_state.tolist()),
            "action": action,
            "reward": reward,
            "done": done,
            "invalid_actions": invalid_actions,
            "next_invalid_actions": next_invalid_actions,
        }
        self.remote_memory.add(batch)
        return {}

    # 状態における強化学習の可視化用です。
    # 今回ですと、Qテーブルを表示しています。
    def render(self, state: np.ndarray, invalid_actions: List[int], env: EnvForRL) -> None:
        q = self.parameter.get_action_values(state, invalid_actions)
        maxa = np.argmax(q)
        for a in range(self.config.nb_actions):
            if a == maxa:
                s = "*"
            else:
                s = " "
            s += f"{env.action_to_str(a)}: {q[a]:7.5f}"
            print(s)


# RLTrainer

---
学習を定義する部分です。  
RemoteMemory から経験を受け取ってParameterを更新します。  


In [8]:
from srl.base.rl.base import RLTrainer

class MyTrainer(RLTrainer):

    # init の引数は親クラスにそのまま渡します。
    def __init__(self, *args):
        super().__init__(*args)

        # 上で定義した config,parameter,memory が入ります
        self.config = cast(MyConfig, self.config)
        self.parameter = cast(MyParameter, self.parameter)
        self.remote_memory = cast(MyRemoteMemory, self.remote_memory)

        self.train_count = 0

    # (abstractmethod) 学習回数を返します
    def get_train_count(self):
        return self.train_count

    # (abstractmethod)
    def train(self):

        # memoryから経験を取得する
        batchs = self.remote_memory.sample()
        td_error = 0
        for batch in batchs:
            # 各batch毎にQテーブルを更新する

            s = batch["state"]
            n_s = batch["next_state"]
            action = batch["action"]
            reward = batch["reward"]
            done = batch["done"]
            invalid_actions = batch["invalid_actions"]
            next_invalid_actions = batch["next_invalid_actions"]

            q = self.parameter.get_action_values(s, invalid_actions, False)
            n_q = self.parameter.get_action_values(n_s, next_invalid_actions, False)

            if done:
                target_q = reward
            else:
                target_q = reward + self.config.gamma * max(n_q)

            td_error = target_q - q[action]
            q[action] += self.config.lr * td_error

            td_error += td_error
            self.train_count += 1

        if len(batchs) > 0:
            td_error /= len(batchs)
        
        # 学習結果の情報を返す
        return {
            "Q": len(self.parameter.Q),
            "td_error": td_error,
        }


# 登録

最後にこれらのクラスを登録します。

In [9]:
from srl.base.rl.registration import register
register(
    MyConfig,
    __name__ + ":MyRemoteMemory",
    __name__ + ":MyParameter",
    __name__ + ":MyTrainer",
    __name__ + ":MyWorker",
)


実行

In [10]:
from srl.runner import sequence
from srl.runner.callbacks import PrintProgress

config = sequence.Config(
    env_name="Grid",
    rl_config=MyConfig(),
)

# --- train
config.set_play_config(max_episodes=10000, training=True, callbacks=[PrintProgress()])
parameter, remote_memory = sequence.train(config)



### env: Grid, max episodes: 10000, max steps: -1, timeout:  -1.00s
20:43:43   5.00s    7054ep   52990tr   1.65s(remain), -2.720 0.689 0.840 reward, 7.5 step, 0.00s/ep, 0.0000s/tr,        0 mem|Q 10.998|td_error 0.017
20:43:46   5.00s   10000ep   75649tr   0.00s(remain), -2.360 0.678 0.840 reward, 7.7 step, 0.00s/ep, 0.0000s/tr,        0 mem|Q 11.000|td_error 0.014


In [11]:
config.set_play_config(max_episodes=100)
rewards, _, _ = sequence.play(config, parameter)
print("100エピソードの平均結果", np.mean(rewards))


100エピソードの平均結果 0.7724000000000001


In [12]:
from srl.runner.callbacks import Rendering

config.set_play_config(max_episodes=1, callbacks=[Rendering(step_stop=False)])
rewards, _, _ = sequence.play(config, parameter)
rewards


### 0
OOOOOO
O   GO
O O XO
OP   O
OOOOOO

 ←: 0.30370
 ↓: 0.27639
 →: 0.22741
*↑: 0.37876
### 1, done: False
player 0, action 3, reward: -0.04
OOOOOO
O   GO
OPO XO
O    O
OOOOOO

env_info  : {}
work_info 0: {}
train_info: None
 ←: 0.40603
 ↓: 0.31480
 →: 0.40661
*↑: 0.51889
### 2, done: False
player 0, action 3, reward: -0.04
OOOOOO
OP  GO
O O XO
O    O
OOOOOO

env_info  : {}
work_info 0: {}
train_info: None
 ←: 0.50961
 ↓: 0.45594
*→: 0.63925
 ↑: 0.53670
### 3, done: False
player 0, action 2, reward: -0.04
OOOOOO
O P GO
O O XO
O    O
OOOOOO

env_info  : {}
work_info 0: {}
train_info: None
 ←: 0.53697
 ↓: 0.65462
*→: 0.75578
 ↑: 0.64757
### 4, done: False
player 0, action 2, reward: -0.04
OOOOOO
O  PGO
O O XO
O    O
OOOOOO

env_info  : {}
work_info 0: {}
train_info: None
 ←: 0.64625
 ↓: 0.51915
*→: 0.92625
 ↑: 0.77844
### 5, done: False
player 0, action 2, reward: -0.04
OOOOOO
O  PGO
O O XO
O    O
OOOOOO

env_info  : {}
work_info 0: {}
train_info: None
 ←: 0.64625
 ↓: 0.51915
*→: 0.926

[0.8]